In [0]:
import re
from pyspark.sql.functions import (
    col,            
    lit,           
    when,           
    regexp_replace, 
    to_date,       
    to_timestamp    
)


In [0]:

target_catalog = "silver"
target_schema = "f1_world"

In [0]:
def replace_coluns(text):
    temp = re.sub(r"([a-z])([A-Z])",r"\1_\2",text)
    return temp.lower()


def tratar_nulos(df):
    for coluna in df.columns:
        df = df.withColumn(coluna,
                                    when((col(coluna)) == "\\N" , lit(None)).
                                    otherwise(col(coluna)))
    
    return df


def aplicar_tipagem(df, map_type):

    coluna_final = []

    for coluna_atual in df.columns:
        if coluna_atual in map_type:
            tipo_alvo = map_type[coluna_atual]
            coluna_final.append(col(coluna_atual).cast(tipo_alvo))
        else:
            coluna_final.append(col(coluna_atual))
    return df.select(coluna_final)




In [0]:
# Regras para CIRCUITS
regras_circuits = {
    "circuit_id": "int",
    "circuit_ref": "string",
    "name": "string",
    "location": "string",
    "country": "string",
    "lat": "float",
    "lng": "float",
    "alt": "float"
}

# Regras para DRIVERS
regras_drivers = {
    "driver_id": "int",
    "driver_ref": "string",
    "number": "int",
    "code": "string",
    "forename": "string",
    "surname": "string",
    "dob": "date",
    "nationality": "string"
}

# Regras para RACES
regras_races = {
    "race_id": "int",
    "year": "int",
    "round": "string",
    "circuit_id": "int",
    "name": "string",
    "date": "date",
    "time": "string",
    "url": "string",
    "fp1_date": "date",
    "fp1_time": "string",
    "fp2_date": "date",
    "fp2_time": "string",
    "fp3_date": "date",
    "fp3_time": "string",
    "quali_date": "date",
    "quali_time": "string",
    "sprint_date": "date",
    "sprint_time": "string",
}

# Regras para RESULTS
regras_results = {
    "result_id": "int",
    "race_id": "int",
    "driver_id": "int",
    "constructor_id": "int",
    "number": "int",
    "grid": "int",
    "position": "int",
    "position_text": "string",
    "position_order": "string",
    "points": "float",
    "laps": "string",
    "time": "string",
    "milliseconds": "int",
    "fastest_lap": "string",
    "rank": "string",
    "fastest_lap_time": "float",
    "fastest_lap_speed": "float",
    "status_id": "int",
}

# Regras para CONSTRUCTORS
regras_constructors = {
    "constructor_id": "int",
    "constructor_ref": "string",
    "name": "string",
    "nationality": "string"
}

# Regras para QUALIFYING
regras_qualifying = {
    "qualify_id": "int",
    "race_id": "int",
    "driver_id": "int",
    "constructor_id": "int",
    "number": "int",
    "position": "int",
    "q1": "string",
    "q2": "string",
    "q3": "string",
}

# Regras para PIT_STOPS
regras_pit_stops = {
    "race_id": "int",
    "driver_id": "int",
    "stop": "string",
    "lap": "int",
    "time": "string",
    "duration": "string",
    "milliseconds": "int",
}

# Regras para CONSTRUCTOR_RESULTS
regras_constructor_results = {
    "constructor_results_id": "int",
    "race_id": "int",
    "constructor_id": "int",
    "points": "float",
    "status": "string",
}

# Regras para CONSTRUCTOR_STANDINGS
regras_constructor_standings = {
    "constructor_standings_id": "int",
    "race_id": "int",
    "constructor_id": "int",
    "points": "float",
    "position": "int",
    "position_text": "string",
    "wins": "string",
}

# Regras para DRIVER_STANDINGS
regras_driver_standings = {
    "driver_standings_id": "int",
    "race_id": "int",
    "driver_id": "int",
    "points": "float",
    "position": "int",
    "position_text": "string",
    "wins": "string",
}

# Regras para LAP_TIMES
regras_lap_times = {
    "race_id": "int",
    "driver_id": "int",
    "lap": "int",
    "position": "int",
    "time": "string",
    "milliseconds": "int",
}

# Regras para STATUS
regras_status = {
    "status_id": "int",
    "status": "string",
}

# Regras para SPRINT_RESULTS
regras_sprint_results = {
    "result_id": "int",
    "race_id": "int",
    "driver_id": "int",
    "constructor_id": "int",
    "number": "int",
    "grid": "int",
    "position": "int",
    "position_text": "string",
    "position_order": "string",
    "points": "float",
    "laps": "string",
    "time": "string",
    "milliseconds": "int",
    "fastest_lap": "string",
    "fastest_lap_time": "float",
    "status_id": "int",
}

# Regras para SEASONS
regras_seasons = {
    "year": "int",
}


map_type = [
    regras_circuits,
    regras_drivers,
    regras_races,
    regras_results,
    regras_constructors,
    regras_qualifying,
    regras_pit_stops,
    regras_constructor_results,
    regras_constructor_standings,
    regras_driver_standings,
    regras_lap_times,
    regras_status,
    regras_sprint_results,
    regras_seasons]


mapa_tabelas = {
    "circuits":       {"regras": regras_circuits,      "particao": None},
    "races":          {"regras": regras_races,         "particao": None},
    "constructors":   {"regras": regras_constructors,  "particao": None},
    "drivers":        {"regras": regras_drivers,       "particao": None},
    "lap_times":      {"regras": regras_lap_times,     "particao": "race_id"},
    "results":        {"regras": regras_results,       "particao": "race_id"},
    "pit_stops":      {"regras": regras_pit_stops,     "particao": "race_id"},
    "constructor_results": {"regras": regras_constructor_results, "particao": "race_id"},
    "constructor_standings": {"regras": regras_constructor_standings, "particao": "race_id"},
    "driver_standings": {"regras": regras_driver_standings, "particao": "race_id"},
    "status":         {"regras": regras_status,        "particao": None},
    "seasons":     {"regras": regras_seasons,    "particao": None},
    "sprint_results":     {"regras": regras_sprint_results,    "particao": "race_id"},
    
}


In [0]:
# Mapeamento


force_update = True

print("🚀 Iniciando Processamento Silver Batch...")

for tabela, config in mapa_tabelas.items():

    if force_update or not spark.catalog.tableExists(f"{target_catalog}.{target_schema}.{tabela}"):
        try:
            print(f"Processando: {tabela.upper()}...")
            # 1. Leitura
            df = spark.table(f"bronze.f1_world.{tabela}")
            
            # 2. Padronização (Regex)
            cols_novas = [col(c).alias(replace_coluns(c)) for c in df.columns]
            df = df.select(*cols_novas)
            
            # 3. Limpeza e Tipagem
            df = tratar_nulos(df)
            df = aplicar_tipagem(df, config["regras"])

            colunas_para_remover = ["url"]

            # Dropa apenas se a coluna existir no dataframe
            for c in colunas_para_remover:
                if c in df.columns:
                    df = df.drop(c)
            
            # 4. Escrita
            spark.sql(f"CREATE SCHEMA IF NOT EXISTS {target_catalog}.{target_schema}")
            writer = df.write.format("delta").mode("overwrite")
            if config["particao"]:
                writer = writer.partitionBy(config["particao"])
                
            writer.saveAsTable(f"{target_catalog}.{target_schema}.{tabela}")
            print(f"✅ {tabela} OK!")
            
        except Exception as e:
            print(f"❌ Erro em {tabela}: {str(e)}")
    else:
        print(f"⏭️  Skipping {tabela} (Tabela já existe e force_update=False)")